In [ ]:
import sys
sys.path.append('E:\\Code\\CompSci_rp\\CompSci\\01_regression_resampling\\src\\')
sys.path.append('E:\\Code\\CompSci_rp\\CompSci\\01_regression_resampling\\misc\\')
import plots
import linear_model as lin 
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
plt.style.use('E:\\Code\\CompSci_rp\\CompSci\\01_regression_resampling\\misc\\report.mplstyle')
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable



data = load_breast_cancer()
X = data.data
y = data.target

#Is there a way to get the train test split used in the StochasticGradientDescent class?
train_ids = np.sort(np.random.choice(len(X),size=450,replace=False))
X_train, y_train = X[train_ids], y[train_ids]
X_test  = X[np.sort(np.setdiff1d(np.arange(0,len(X)),train_ids))]
y_test = y[np.sort(np.setdiff1d(np.arange(0,len(X)),train_ids))]


In [ ]:
#%% Extract the features present in the Data with PCA
from sklearn.decomposition import PCA

features_cor = np.corrcoef(data.data.T)
features_PCA = PCA().fit(data.data)
features_decomp = PCA().fit_transform(data.data)
exp_var = features_PCA.explained_variance_ratio_


In [ ]:
fig_size = plots.set_size(1000)
fig = plt.figure(figsize=fig_size)
gs = gridspec.GridSpec(6,12)
ax_1 = plt.subplot(gs[:6,:6])
ax1=ax_1.imshow(features_cor,cmap='coolwarm')
plt.xticks(np.arange(0,len(features_cor)),labels=data.feature_names,rotation=90)
plt.colorbar(ax1,fraction=0.046,pad=0.04)
ax_2 = plt.subplot(gs[:4,6:])
ax_2.plot(features_decomp[y==0,0],features_decomp[y==0,1],'o')
ax_2.plot(features_decomp[y==1,0],features_decomp[y==1,1],'o')
ax_2.legend(['benign','malignant'])
ax_2.set_xlabel('PC1')
ax_2.set_ylabel('PC2')
ax_3 = plt.subplot(gs[4:,6:])
ax_3.plot(exp_var,'r-o')
# ax_3.set_xlabel('PC #')
ax_3.set_ylabel('explained variance ratio')
plt.tight_layout()

In [ ]:
#%% Sklearn implementation vs Support Vector Machines vs 
#our implementation across learning rates and regularization values

#set pca_reduction to an integer corresponding to the number of principal features
#to do classification with. 0 leads to the use of the original features
pca_reduction = 0
if pca_reduction>0:        
    X_train = features_decomp[train_ids,:pca_reduction]
    X_test = features_decomp[np.sort(np.setdiff1d(np.arange(0,len(X)),train_ids)),:pca_reduction]
lrs = np.logspace(-6,0,7)
penalties = np.logspace(-9,0,10)
success_rate_ours = np.zeros([len(penalties),len(lrs)])
success_rate_sklearn = np.zeros([len(penalties),len(lrs)])
success_rate_svm = np.zeros([len(penalties)])
count = 0

for p in range(len(penalties)):
    Classifier_SVC = SVC(C=1/penalties[p])
    fit_svc = Classifier_SVC.fit(X_train,y_train)
    for l in range(len(lrs)):
        Classifier_sklearn = SGDClassifier(loss='log',penalty='l2',alpha=penalties[p],
                                           learning_rate='constant',eta0=lrs[l],verbose=42)
        Classifier_ours = lin.StochasticGradientDescent(loss='cross_entropy',batches=10, 
                                            learning_rate= lambda t, grad: lrs[l],momentum=0.,
                                            early_stopping=False,validation_fraction=0.1,seed=42,
                                            l2_penalty=penalties[p])
        fit_ours, fit_sklearn = Classifier_ours.fit(X_train,y_train), Classifier_sklearn.fit(X_train,y_train)
        params_ours, params_sklearn  = Classifier_ours.params, Classifier_sklearn.get_params()
        classified = expit(X_test@params_ours)
        classified[classified>0.5] = 1
        classified[classified<0.5] = 0
        success_rate_ours[p,l] = np.sum(classified==y_test)/len(y_test)
        success_rate_sklearn[p,l] = Classifier_sklearn.score(X_test,y_test)
    success_rate_svm[p] = Classifier_SVC.score(X_test,y_test)
success_rate_svm = np.expand_dims(success_rate_svm,1)    

In [ ]:
#%%Plot of the performance of different methods
plt.figure(dpi=200)
plt.subplot(1,3,1)
plots.annotate_imshow(success_rate_ours,vmin=0.5)
plt.xticks(np.arange(0,len(lrs)),lrs,rotation=45)
plt.yticks(np.arange(0,len(penalties)),penalties)
plt.title('Our implementation')
plt.subplot(1,3,2)
plots.annotate_imshow(success_rate_sklearn,vmin=0.5)
plt.xticks(np.arange(0,len(lrs)),lrs,rotation=45)
plt.yticks(np.arange(0,len(penalties)),penalties)
plt.title('Sklearn')
plt.subplot(1,3,3)
plots.annotate_imshow(success_rate_svm,vmin=0.5)
plt.yticks(np.arange(0,len(penalties)),penalties)
plt.xticks([])
plt.title('SVM')
plt.tight_layout()